## Solving the Zebra Puzzle with Z3

Puzzle statment: https://en.wikipedia.org/wiki/Zebra_Puzzle

In [1]:
import z3

ENG, SPA, UKR, NOR, JAP = range(5)
nat_names = ["Englishman", "Spaniard", "Ukrainian", "Norweigian", "Japanese"]
RED, GREEN, IVORY, YELLOW, BLUE = range(5)
clr_names = ["red", "green", "ivory", "yellow", "blue"]
DOG, SNAIL, FOX, HORSE, ZEBRA = range(5)
anm_names = ["dog", "snails", "fox", "horse", "zebra"]
COFFEE, TEA, MILK, JUICE, WATER = range(5)
bev_names = ["coffee", "tea", "milk", "juice", "water"]
OG, KOOLS, CF, LS, PARL = range(5)
solver = z3.Solver()

# 1.There are five houses.
def make_vars(var_name):
  v = []
  for i in range(5):
    v.append(z3.Int(f"{var_name}-{i}"))
    solver.add(v[i] >= 0)
    solver.add(v[i] <= 4)
  solver.add(z3.Distinct(*v))
  return v
nat = make_vars("nat")
clr = make_vars("clr")
anm = make_vars("anm")
bev = make_vars("bev")
cig = make_vars("cig")

# 2.The Englishman lives in the red house.
for i in range(5):
  solver.add(z3.Implies(nat[i]==ENG, clr[i]==RED))

# 3.The Spaniard owns the dog.
for i in range(5):
  solver.add(z3.Implies(nat[i]==SPA, anm[i]==DOG))

# 4.Coffee is drunk in the green house.
for i in range(5):
  solver.add(z3.Implies(bev[i]==COFFEE, clr[i]==GREEN))

# 5.The Ukrainian drinks tea.
for i in range(5):
  solver.add(z3.Implies(nat[i]==UKR, bev[i]==TEA))

# 6.The green house is immediately to the right of the ivory house.
for i in range(4):
  solver.add(z3.Implies(clr[i]==IVORY, clr[i+1]==GREEN))

# 7.The Old Gold smoker owns snails.
for i in range(5):
  solver.add(z3.Implies(cig[i]==OG, anm[i]==SNAIL))

# 8.Kools are smoked in the yellow house.
for i in range(5):
  solver.add(z3.Implies(clr[i]==YELLOW, cig[i]==KOOLS))

# 9.Milk is drunk in the middle house.
solver.add(bev[2]==MILK)

# 10.The Norwegian lives in the first house.
solver.add(nat[0]==NOR)

# 11.The man who smokes Chesterfields lives in the house next to the man with the fox.
def neighbors(cond1, cond2):
  or_terms = []
  for i in range(4):
    or_terms.append(z3.And(cond1(i), cond2(i+1)))
    or_terms.append(z3.And(cond2(i), cond1(i+1)))
  return z3.Or(or_terms)  

solver.add(neighbors(lambda i: cig[i]==CF, lambda j: anm[j]==FOX))

# 12.Kools are smoked in the house next to the house where the horse is kept.
solver.add(neighbors(lambda i: cig[i]==KOOLS, lambda j: anm[j]==HORSE))

# 13.The Lucky Strike smoker drinks orange juice.
for i in range(5):
  solver.add(z3.Implies(cig[i]==LS, bev[i]==JUICE))

# 14.The Japanese smokes Parliaments.
for i in range(5):
  solver.add(z3.Implies(nat[i]==JAP, cig[i]==PARL))

# 15.The Norwegian lives next to the blue house.
solver.add(neighbors(lambda i: nat[i]==NOR, lambda j: clr[j]==BLUE))



print(solver.check())

m = solver.model()
for i in range(5):
  print("House #%d (%s): %s, %s, %s, cig=%d" % (
    i, 
    clr_names[m.eval(clr[i]).as_long()],
    nat_names[m.eval(nat[i]).as_long()],
    anm_names[m.eval(anm[i]).as_long()],
    bev_names[m.eval(bev[i]).as_long()],
    m.eval(cig[i]).as_long()
  ))
  
# Who drinks water?
for i in range(5):
  if m.eval(bev[i]).as_long()==WATER:
    print("%s drinks water." % nat_names[m.eval(nat[i]).as_long()])

# Who owns the zebra?
for i in range(5):
  if m.eval(anm[i]).as_long()==ZEBRA:
    print("%s owns the zebra." % nat_names[m.eval(nat[i]).as_long()])

sat
House #0 (yellow): Norweigian, fox, water, cig=1
House #1 (blue): Ukrainian, horse, tea, cig=2
House #2 (red): Englishman, snails, milk, cig=0
House #3 (ivory): Spaniard, dog, juice, cig=3
House #4 (green): Japanese, zebra, coffee, cig=4
Norweigian drinks water.
Japanese owns the zebra.
